In [1]:
import os
import glob
from PIL import Image
from pix2tex.cli import LatexOCR


def get_most_recent(folder, extension="*"):
    patron = os.path.join(folder, f"*.{extension}")
    file_list = glob.glob(patron)
    if not file_list:
        return None
    latest_file = max(file_list, key=os.path.getmtime)
    return os.path.abspath(latest_file)


path = "/home/hollman/Pictures/Screenshots/"
latest = get_most_recent(path)

if latest:
    print(f"El archivo más reciente es: {latest}")
else:
    print("No se encontraron archivos en la carpeta especificada.")

img = Image.open(latest)
model = LatexOCR()
TEXT = str(model(img))
print(model(img))



/home/holman/miniconda3/envs/cosmos/lib/python3.11/site-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.8 (you have 1.4.24). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
/home/holman/miniconda3/envs/cosmos/lib/python3.11/site-packages/pydantic/main.py:426: UserWarning: Pydantic serializer warnings:
  Expected `dict[str, any]` but got `UniformParams` with value `UniformParams(noise_type=... 0.058823529411764705)])` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


No se encontraron archivos en la carpeta especificada.


AttributeError: 'NoneType' object has no attribute 'read'

In [2]:
!pip install feedparser

In [1]:
import feedparser

def formatear_autor(nombre_completo):
    """
    Toma el nombre completo y retorna un string en formato "Inicial.Apellido".
    Se asume que el apellido es la última palabra y el primer nombre es la primera palabra.
    """
    partes = nombre_completo.split()
    if len(partes) >= 2:
        inicial = partes[0][0]  # Primera letra del primer nombre
        apellido = partes[-1]
        return f"{inicial}.{apellido}"
    else:
        # Si no se puede separar, retorna el nombre completo
        return nombre_completo

def obtener_info_arxiv(codigo):
    # Elimina el prefijo 'arXiv:' si está presente
    if codigo.lower().startswith("arxiv:"):
        codigo = codigo[6:]
    
    # Construimos la URL para consultar la API de arXiv
    url = f"http://export.arxiv.org/api/query?id_list={codigo}"
    
    # Parseamos la respuesta del API
    feed = feedparser.parse(url)
    
    # Si no se encontró ningún artículo, se devuelve un mensaje de error
    if not feed.entries:
        return "No se encontró ningún artículo con ese código."
    
    entry = feed.entries[0]
    
    # Título del artículo (limpiando saltos de línea y espacios extra)
    titulo = entry.title.strip().replace('\n', ' ')
    
    # Se obtiene el año de publicación a partir del campo 'published'
    anio = entry.published[:4]
    
    # Se procesan los nombres de los autores
    autores_brutos = [author.name for author in entry.authors]
    autores_formateados = [formatear_autor(nombre) for nombre in autores_brutos]
    
    if len(autores_formateados) > 2:
        # Si son más de dos, se muestran los dos primeros y et al.
        autores_final = ", ".join(autores_formateados[:2]) + ", et al."
    else:
        # Si hay dos o menos, se muestran todos separados por coma
        autores_final = ", ".join(autores_formateados)
    
    # Se arma el texto final
    resultado = (
        f"Autores: {autores_final}\n"
        f"Año: {anio}\n"
        f"Título: {titulo}\n"
        f"Código arXiv: {codigo}"
        f"\n"
        f"Text:  {autores_final},{anio},{titulo},{codigo}"
    )
    
    return resultado



In [2]:
if __name__ == '__main__':
    codigo_usuario = input("Ingresa el código arXiv (ejemplo: arXiv:2101.00001): ")
    info = obtener_info_arxiv(codigo_usuario)
    print(info)

Autores: E.Telles, J.Melnick
Año: 2018
Título: The Stellar Populations of HII galaxies: A tale of three bursts
Código arXiv: 1803.02778
Text:  E.Telles, J.Melnick,2018,The Stellar Populations of HII galaxies: A tale of three bursts,1803.02778


In [4]:
print(r'Primack, J.,2024,Galaxy Formation in $\Lambda$CDM Cosmology.')

Primack, J.,2024,Galaxy Formation in $\Lambda$CDM Cosmology.


In [5]:
import os
import requests

ADS_API_URL = "https://api.adsabs.harvard.edu/v1/search/query"

def formatear_autor(nombre_completo):
    """
    Intenta dejar el autor en formato 'Inicial.Apellido'.

    Soporta tanto:
    - 'Nombre Apellido'
    - 'Apellido, Nombre'
    como los devuelve usualmente ADS.
    """
    # Caso típico de ADS: 'Apellido, Nombre ...'
    if "," in nombre_completo:
        apellido, resto = [x.strip() for x in nombre_completo.split(",", 1)]
        if resto:
            inicial = resto[0]
            return f"{inicial}.{apellido}"
        else:
            return apellido

    # Caso simple tipo 'Nombre Apellido'
    partes = nombre_completo.split()
    if len(partes) >= 2:
        inicial = partes[0][0]
        apellido = partes[-1]
        return f"{inicial}.{apellido}"
    else:
        return nombre_completo


def obtener_info_ads(codigo):
    """
    Consulta NASA ADS usando el campo 'identifier',
    que incluye bibcode, DOI, arxiv id, etc. :contentReference[oaicite:1]{index=1}

    'codigo' puede ser, por ejemplo:
        - un bibcode: '2011ApJ...730..119R'
        - un arXiv id: 'arXiv:1101.12345' o '1101.12345'
        - un DOI, etc.
    """

    token = os.environ.get("ADS_DEV_KEY")
    if not token:
        return (
            "Error: no se encontró el token de ADS.\n"
            "Define la variable de entorno ADS_DEV_KEY con tu API token de ADS."
        )

    # Si viene con prefijo 'arXiv:' lo limpiamos para que sea más flexible
    if codigo.lower().startswith("arxiv:"):
        codigo_limpio = codigo[6:]
    else:
        codigo_limpio = codigo

    headers = {
        "Authorization": f"Bearer {token}"
    }

    # Buscamos en el campo 'identifier', que incluye arXiv IDs, DOIs, bibcodes, etc.
    params = {
        "q": f"identifier:{codigo_limpio}",
        "fl": "title,author,year,identifier,bibcode",
        "rows": 1
    }

    try:
        resp = requests.get(ADS_API_URL, headers=headers, params=params, timeout=15)
    except requests.exceptions.RequestException as e:
        return f"Error de conexión con ADS: {e}"

    if resp.status_code != 200:
        return f"Error en la respuesta de ADS (status {resp.status_code}): {resp.text}"

    data = resp.json()
    docs = data.get("response", {}).get("docs", [])

    if not docs:
        return "No se encontró ningún artículo con ese identificador en ADS."

    doc = docs[0]

    # Título (ADS devuelve una lista de títulos)
    titulos = doc.get("title", [])
    titulo = titulos[0].strip().replace("\n", " ") if titulos else "Título no disponible"

    # Año
    anio = str(doc.get("year", "----"))

    # Autores
    autores_brutos = doc.get("author", [])
    autores_formateados = [formatear_autor(nombre) for nombre in autores_brutos]

    if len(autores_formateados) > 2:
        autores_final = ", ".join(autores_formateados[:2]) + ", et al."
    else:
        autores_final = ", ".join(autores_formateados)

    # Elegimos un identificador "bonito" para mostrar:
    bibcode = doc.get("bibcode", "")
    # Si quieres, podrías tratar de localizar explícitamente el arxiv-id dentro de identifier[]
    # pero para este ejemplo usamos el bibcode como identificador principal.
    identificadores = doc.get("identifier", [])
    identificador_mostrado = bibcode or (identificadores[0] if identificadores else codigo_limpio)

    resultado = (
        f"Autores: {autores_final}\n"
        f"Año: {anio}\n"
        f"Título: {titulo}\n"
        f"Identificador (ADS/bibcode): {identificador_mostrado}"
        f"\n"
        f"Text:  {autores_final},{anio},{titulo},{identificador_mostrado}"
    )

    return resultado


if __name__ == '__main__':
    codigo_usuario = input("Ingresa el identificador (bibcode, arXiv, DOI, etc.): ")
    info = obtener_info_ads(codigo_usuario)
    print(info)


Error: no se encontró el token de ADS.
Define la variable de entorno ADS_DEV_KEY con tu API token de ADS.
